In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [14]:
def get_bins(df, column, n_bins):
    min_v = df[column].min()
    max_v = df[column].max()
    array = []
    
    for i in range(n_bins+1):
        step = max_v/n_bins
        array.append(int(round(step*i, 0)))
    
    return array

In [15]:
## Get the names and open data
data_domicilio_path = ('./data/CENSO/Domicilio02_PB.csv') 

with open(data_domicilio_path, "r", encoding="latin-1") as myfile:
    for i in range(1):
        line = next(myfile).strip()
        names = line.split(';')


data = pd.read_csv(data_domicilio_path,                  
                sep=';', 
                header=None, 
                names=names,
                skiprows=1,
                encoding="latin-1",
                dtype={'Cod_setor': str})

In [16]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [17]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [18]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
moradores = data[['Cod_setor', 'V001']]
moradores = moradores.rename(columns={'V001': 'MORADORES'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [19]:
# Set the same inde
moradores.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [20]:
setores = setores.join(moradores)

In [21]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [22]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [11]:
def get_area(row):
    return row['geometry'].area/10000

def get_density(row):
    density = row['MORADORES']/row['area']
    return density

setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)
bairros['area'] = bairros.apply(get_area, axis=1)
bairros['densidade'] = bairros.apply(get_density, axis=1)

<ipython-input-11-a00a87234b51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
<ipython-input-11-a00a87234b51>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)


In [12]:
setores_recorte.to_file('./data/POPULACAO/setores_recorte.shp')
bairros.to_file('./data/POPULACAO/bairros.shp')